# FAISS 성능 테스트

In [30]:
# !pip install psutil

In [31]:
import os
import pickle
import faiss

# 프로젝트 루트 디렉토리 설정
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

# FAISS 인덱스 및 메타데이터 저장 경로
vector_db_path = os.path.join(BASE_DIR, "data", "db", "faiss_v2")
index_path = os.path.join(vector_db_path, "index.faiss")
metadata_path = os.path.join(vector_db_path, "index.pkl")

def load_faiss_index():
    """
    저장된 FAISS 인덱스 및 메타데이터를 불러오는 함수
    """
    # FAISS 인덱스 확인 및 로드
    if not os.path.exists(index_path):
        raise FileNotFoundError(f"FAISS 인덱스 파일을 찾을 수 없습니다: {index_path}")

    index = faiss.read_index(index_path)
    print("FAISS 인덱스 로드 완료!")

    # 메타데이터 확인 및 로드
    if not os.path.exists(metadata_path):
        raise FileNotFoundError(f"메타데이터 파일을 찾을 수 없습니다: {metadata_path}")

    with open(metadata_path, "rb") as f:
        metadata_list = pickle.load(f)

    print(f"FAISS 인덱스 및 메타데이터 로드 완료! (총 {len(metadata_list)} 개)")
    return index, metadata_list

# FAISS 인덱스 및 메타데이터 로드 실행
index_ivf, metadata = load_faiss_index()


FAISS 인덱스 로드 완료!
FAISS 인덱스 및 메타데이터 로드 완료! (총 2 개)


In [32]:
# !pip install sentence-transformers

In [33]:
import time
import psutil
import numpy as np
import faiss
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# OpenAI Embeddings 로드
embeddings = OpenAIEmbeddings()

def evaluate_faiss_performance(vectorstore, query_text, top_k=5):
    """
    FAISS 검색 성능 평가 (검색 속도, Recall@K, 메모리 사용량)
    """
    # 쿼리를 벡터로 변환
    query_embedding = embeddings.embed_query(query_text)
    query_embedding = np.array(query_embedding, dtype="float32").reshape(1, -1)

    # 검색 속도 측정
    start_time = time.perf_counter()
    distances, indices = vectorstore.index.search(query_embedding, top_k)
    end_time = time.perf_counter()
    search_time = (end_time - start_time) * 1000  # ms 변환

    # 검색된 인덱스 값 확인
    print("검색된 indices:", indices)
    print("metadata 길이:", len(vectorstore.docstore._dict))  

    # 검색 결과 정리
    results = []
    for i in range(top_k):
        idx = indices[0][i]
        
        # 인덱스가 유효한 범위인지 확인
        if str(idx) not in vectorstore.docstore._dict:  
            continue  

        matched_text = vectorstore.docstore._dict[str(idx)]["output"] 
        score = distances[0][i]
        results.append({"rank": i+1, "score": score, "output": matched_text})

    # Recall@K 계산 (IVF 인덱스 vs 정확한 IndexFlatL2 비교)
    index_flat = faiss.IndexFlatL2(vectorstore.index.d)
    index_flat.add(vectorstore.index.reconstruct_n(0, vectorstore.index.ntotal))
    true_distances, true_indices = index_flat.search(query_embedding, top_k)

    true_set = set(true_indices[0])
    faiss_set = set(indices[0])
    recall_k = len(true_set & faiss_set) / top_k

    # 메모리 사용량 확인
    process = psutil.Process()
    memory_used = process.memory_info().rss / (1024 * 1024)  # MB 단위 변환

    # 평가 결과 출력
    print(f"\n검색어: {query_text}")
    print(f"검색 속도: {search_time:.3f} ms")
    print(f"Recall@K(정확도): {recall_k:.2f}")
    print(f"메모리 사용량: {memory_used:.2f} MB")
    print("\n검색 결과 (Top 5):")
    for res in results:
        print(f" - Rank {res['rank']} | Score: {res['score']:.4f} | {res['output']}")

    return {
        "search_time_ms": search_time,
        "recall_k": recall_k,
        "memory_usage_mb": memory_used,
        "results": results
    }

# 검색 평가 실행
query_text = "청소년이 우울할 때 어떻게 해야 하나요?"

# FAISS 인덱스 로드
vector_db_path = "../../data/db/faiss_v2"
vectorstore = FAISS.load_local(vector_db_path, embeddings, allow_dangerous_deserialization=True)

# 성능 평가 실행
evaluation_result = evaluate_faiss_performance(vectorstore, query_text)


검색된 indices: [[1966  705  509  575 8237]]
metadata 길이: 13234

검색어: 청소년이 우울할 때 어떻게 해야 하나요?
검색 속도: 0.704 ms
Recall@K(정확도): 0.80
메모리 사용량: 590.98 MB

검색 결과 (Top 5):


- 검색 속도: 0.704 ms
- Recall@K(정확도): 0.80
- 메모리 사용량: 590.98 MB

1. 1ms 이하는 거의 즉시 검색 수준
    - FAISS의 IVF 인덱스 최적화가 잘 적용된 상태임.
    - 전체 데이터(13,234개) 중에서 유사한 클러스터만 빠르게 탐색하여 속도가 빠름(0.1~1ms면 매우 빠른 편)
2. Recall@K가 0.80이면, 검색된 5개 결과 중 4개가 정확한 Ground Truth와 일치한다는 의미이므로 좋은 성능. 
    - 더 높이면 검색 속도가 느려질 수 있음
    - nprobe=10 설정이 속도와 정확도의 균형을 잘 맞춘 상태라고 생각함
3. FAISS IVF 인덱스 기준으로 적절한 메모리 사용량
    - OpenAI Embeddings (1536차원)이 기본적으로 고차원 벡터이므로 벡터 데이터 저장에 메모리를 많이 사용할 수밖에 없음
    - 기본적으로 벡터 자체 크기가 크기 때문에 어느 정도 메모리를 사용해야 함